In [1]:
!pip install datasets transformers
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:0000:01


In [2]:
from huggingface_hub import notebook_login
from evaluate import load

notebook_login()

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator
import shutil
import os
import glob
from collections import defaultdict
import collections
import numpy as np
from tqdm.auto import tqdm

In [4]:
from datasets import load_dataset



langs = ["de","en","es","hi"]

translate_langs = ["de","es","hi"]

langs_test = []

langs_translate_test = []

langs_translate_train = ["de.de","en.en","es.es","hi.hi"]

langs_translate_train_all = []



mlqa = {}



for lang1 in langs:

    for lang2 in langs:

        mlqa[f"{lang1}.{lang2}"] = load_dataset("mlqa", f"mlqa.{lang1}.{lang2}")

        langs_test.append(f"{lang1}.{lang2}")



for lang in translate_langs:

    mlqa[f"translate-train.{lang}"] = load_dataset("mlqa", f"mlqa-translate-train.{lang}")

    mlqa[f"translate-test.{lang}"] = load_dataset("mlqa", f"mlqa-translate-test.{lang}")

    langs_translate_test.append(f"translate-test.{lang}")

    langs_translate_train_all.append(f"translate-train.{lang}")

mlqa.py:   0%|          | 0.00/8.44k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/34.9k [00:00<?, ?B/s]

The repository for mlqa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mlqa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating test split:   0%|          | 0/4517 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/512 [00:00<?, ? examples/s]

...

Generating validation split:   0%|          | 0/10123 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5253 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/82451 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10253 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4918 [00:00<?, ? examples/s]

In [5]:
mlqa

{'de.de': DatasetDict({
     test: Dataset({
         features: ['context', 'question', 'answers', 'id'],
         num_rows: 4517
     })
     validation: Dataset({
         features: ['context', 'question', 'answers', 'id'],
         num_rows: 512
     })
 }),
 'de.en': DatasetDict({
     test: Dataset({
         features: ['context', 'question', 'answers', 'id'],
         num_rows: 4517
     })
     validation: Dataset({
         features: ['context', 'question', 'answers', 'id'],
         num_rows: 512
     })
 }),
 'de.es': DatasetDict({
     test: Dataset({
         features: ['context', 'question', 'answers', 'id'],
         num_rows: 1776
     })
     validation: Dataset({
         features: ['context', 'question', 'answers', 'id'],
         num_rows: 196
     })
 }),
 'de.hi': DatasetDict({
     test: Dataset({
         features: ['context', 'question', 'answers', 'id'],
         num_rows: 1430
     })
     validation: Dataset({
         features: ['context', 'question', 'answe

In [6]:
mlqa["translate-train.es"]["train"][0]

{'context': 'Arquitectónico, la escuela tiene un carácter católico. sobre la cúpula de oro del edificio principal es una estatua de oro de la Virgen María. inmediatamente frente al edificio principal y frente a ella, es una estatua de cobre de Cristo con los brazos levantado con la leyenda venite ad me omnes. junto al edificio principal se encuentra la Basílica del sagrado corazón. Inmediatamente detrás de la Basílica se encuentra la gruta, un lugar de oración y reflexión de Marian. Se trata de una réplica de la gruta en Lourdes, Francia, donde la Virgen María supuestamente apareció a saint bernadette soubirous en 1858. Al final de la unidad principal (y en una línea directa que conecta a través de 3 estatuas y la cúpula de oro), es una simple y moderna estatua de piedra de María.',
 'question': 'A quién presuntamente apareció la Virgen María en 1858 en Lourdes Francia?',
 'answers': {'answer_start': [575], 'text': ['Santa Bernadette Soubirous']},
 'id': '5733be284776f41900661182'}

In [7]:
mlqa["translate-train.es"]["validation"][0]

{'context': "Super Bowl 50 fue un juego de fútbol americano para determinar el campeón de la liga nacional de fútbol (NFL) para la temporada 2015 la conferencia de fútbol americano (AFC) campeón DENVER BRONCOS derrotó a la conferencia nacional de fútbol (NFC) Campeona Carolina Panthers 24-10 para ganar su tercer título de super bowl. El juego se jugó el 7 de febrero de 2016, en el estadio Levi ' s en la zona de la bahía de San Francisco en santa clara, California. Como este fue el 50º super bowl, la liga hizo hincapié en el aniversario de oro con varias iniciativas temáticas de oro, así como suspender temporalmente la tradición de nombrar cada juego de super bowl con números romanos (bajo el cual el juego habría sido conocido como super bowl l), de modo que el logotipo podría característica de forma prominente los números árabes 50.",
 'question': 'Qué equipo de la nfl representó a la afc en super bowl 50?',
 'answers': {'answer_start': [182, 182, 182],
  'text': ['DENVER BRONCOS', 'DE

In [ ]:
max_length = 384

doc_stride = 128 

In [ ]:
def prepare_train_features(examples):

    
    examples["question"] = [q.lstrip() for q in examples["question"]]



   
    tokenized_examples = tokenizer(

        examples["question"],

        examples["context"],

        truncation="only_second",

        max_length=max_length,

        stride=doc_stride,

        return_overflowing_tokens=True,

        return_offsets_mapping=True,

        padding="max_length",

    )



   
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

  

    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []

    tokenized_examples["end_positions"] = []



    for i, offsets in enumerate(offset_mapping):


        input_ids = tokenized_examples["input_ids"][i]

        cls_index = input_ids.index(tokenizer.cls_token_id)


        sequence_ids = tokenized_examples.sequence_ids(i)


        sample_index = sample_mapping[i]

        answers = examples["answers"][sample_index]

        if len(answers["answer_start"]) == 0:

            tokenized_examples["start_positions"].append(cls_index)

            tokenized_examples["end_positions"].append(cls_index)

        else:

            start_char = answers["answer_start"][0]

            end_char = start_char + len(answers["text"][0])

            token_start_index = 0

            while sequence_ids[token_start_index] != 1:

                token_start_index += 1


            token_end_index = len(input_ids) - 1

            while sequence_ids[token_end_index] != 1:

                token_end_index -= 1



            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):

                tokenized_examples["start_positions"].append(cls_index)

                tokenized_examples["end_positions"].append(cls_index)

            else:

                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:

                    token_start_index += 1

                tokenized_examples["start_positions"].append(token_start_index - 1)

                while offsets[token_end_index][1] >= end_char:

                    token_end_index -= 1

                tokenized_examples["end_positions"].append(token_end_index + 1)



    return tokenized_examples

In [ ]:
def prepare_validation_features(examples):


    examples["question"] = [q.lstrip() for q in examples["question"]]

    tokenized_examples = tokenizer(

        examples["question"],

        examples["context"],

        truncation="only_second",

        max_length=max_length,

        stride=doc_stride,

        return_overflowing_tokens=True,

        return_offsets_mapping=True,

        padding="max_length",

    )


    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")


    tokenized_examples["example_id"] = []



    for i in range(len(tokenized_examples["input_ids"])):

        sequence_ids = tokenized_examples.sequence_ids(i)

        context_index = 1

        sample_index = sample_mapping[i]

        tokenized_examples["example_id"].append(examples["id"][sample_index])

        tokenized_examples["offset_mapping"][i] = [

            (o if sequence_ids[k] == context_index else None)

            for k, o in enumerate(tokenized_examples["offset_mapping"][i])

        ]



    return tokenized_examples

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
from tqdm.auto import tqdm

import collections

import numpy as np



def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):

    all_start_logits, all_end_logits = raw_predictions

    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}

    features_per_example = collections.defaultdict(list)

    for i, feature in enumerate(features):

        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    predictions = collections.OrderedDict()

    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")


    for example_index, example in enumerate(tqdm(examples)):

        feature_indices = features_per_example[example_index]

        valid_answers = []

        context = example["context"]

        for feature_index in feature_indices:

            start_logits = all_start_logits[feature_index]

            end_logits = all_end_logits[feature_index]

            offset_mapping = features[feature_index]["offset_mapping"]


            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()

            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()

            for start_index in start_indexes:

                for end_index in end_indexes:

                    if (

                        start_index >= len(offset_mapping)

                        or end_index >= len(offset_mapping)

                        or offset_mapping[start_index] is None

                        or offset_mapping[end_index] is None

                    ):

                        continue

                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:

                        continue



                    start_char = offset_mapping[start_index][0]

                    end_char = offset_mapping[end_index][1]

                    valid_answers.append(

                        {

                            "score": start_logits[start_index] + end_logits[end_index],

                            "text": context[start_char: end_char]

                        }

                    )

        

        if len(valid_answers) > 0:

            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]

        else:

            best_answer = {"text": "", "score": 0.0}


        predictions[example["id"]] = best_answer["text"]



    return predictions

In [ ]:
from collections import defaultdict

mlqa_prep = defaultdict(dict)

def map_datasets(langs, split, prepare_features):

    for lang in langs:

        mlqa_prep[lang][split] = mlqa[lang][split].map(prepare_features, batched=True, 

                                    remove_columns=mlqa[lang][split].column_names)

In [ ]:
def compute_results(langs, split):

    results = {}

    for lang in langs:

        raw_predictions = trainer.predict(mlqa_prep[lang][split])


        mlqa_prep[lang][split].set_format(type=mlqa_prep[lang][split].format["type"], 

                        columns=list(mlqa_prep[lang][split].features.keys()))

        
        final_predictions = postprocess_qa_predictions(mlqa[lang][split], mlqa_prep[lang][split], raw_predictions.predictions)


        formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]

        references = [{"id": ex["id"], "answers": ex["answers"]} for ex in mlqa[lang][split]]

        results[lang] = metric.compute(predictions=formatted_predictions, references=references)

    return results


import pandas as pd

def results_df(results_dict, model):

    F1colname = "F1_" + model

    EMcolname = "EM_" + model

    dict_results = defaultdict(list)

    for lang, scores in results_dict.items():

        dict_results["lang"].append(lang)

        dict_results[F1colname].append(scores['f1'])

        dict_results[EMcolname].append(scores['exact_match'])



    avg_f1 = np.average(dict_results[F1colname])

    avg_em = np.average(dict_results[EMcolname])

    dict_results["lang"].append('avg')

    dict_results[F1colname].append(avg_f1)

    dict_results[EMcolname].append(avg_em)

    df_results = pd.DataFrame(dict_results).round(2)

    return df_results


# ****Translate Train Es XLM-R****

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, DefaultDataCollator



model_name = "saattrupdan/xlmr-base-texas-squad-es"

model = AutoModelForQuestionAnswering.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name)

data_collator = DefaultDataCollator()



batch_size = 16

training_args = TrainingArguments(

    output_dir="xlmr-base-texas-squad-es",

    evaluation_strategy="epoch",

    learning_rate=2e-5,

    per_device_train_batch_size=batch_size,

    per_device_eval_batch_size=batch_size,

    num_train_epochs=3,

    weight_decay=0.01,

    save_strategy="epoch",            
    
    load_best_model_at_end=True,        
    
    logging_strategy="epoch"         
)

lang = ["translate-train.es"]



split = "train"

map_datasets(lang, split, prepare_train_features)



split = "validation"

map_datasets(lang, split, prepare_train_features)


trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=mlqa_prep["translate-train.es"]["train"],

    eval_dataset=mlqa_prep["translate-train.es"]["validation"],

    tokenizer=tokenizer,

    data_collator=data_collator,

)

trainer.train()

config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

...

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=7983, training_loss=1.5685475495290775, metrics={'train_runtime': 11631.5997, 'train_samples_per_second': 21.955, 'train_steps_per_second': 0.686, 'total_flos': 5.004473068548403e+16, 'train_loss': 1.5685475495290775, 'epoch': 3.0})

In [16]:
metric = load("squad")

split = "test"

map_datasets(langs_translate_train, split, prepare_validation_features)

Map:   0%|          | 0/4517 [00:00<?, ? examples/s]

Map:   0%|          | 0/11590 [00:00<?, ? examples/s]

Map:   0%|          | 0/5253 [00:00<?, ? examples/s]

Map:   0%|          | 0/4918 [00:00<?, ? examples/s]

In [17]:
split = "test"

results_translate_train_es_xlm_r = compute_results(langs_translate_train, split)

print(results_translate_train_es_xlm_r)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


Post-processing 4517 example predictions split into 5278 features.


  0%|          | 0/4517 [00:00<?, ?it/s]

Post-processing 11590 example predictions split into 15269 features.


  0%|          | 0/11590 [00:00<?, ?it/s]

Post-processing 5253 example predictions split into 5457 features.


  0%|          | 0/5253 [00:00<?, ?it/s]

Post-processing 4918 example predictions split into 6351 features.


  0%|          | 0/4918 [00:00<?, ?it/s]

{'de.de': {'exact_match': 45.56121319459818, 'f1': 62.6523863691191}, 'en.en': {'exact_match': 64.09836065573771, 'f1': 79.04587623267263}, 'es.es': {'exact_match': 46.10698648391396, 'f1': 69.34885176860061}, 'hi.hi': {'exact_match': 43.83895892639284, 'f1': 62.6057669117793}}


In [18]:
df_results_translate_train_es_xlm_r = results_df(results_translate_train_es_xlm_r, "Translate-train es XLM-R")

df_results_translate_train_es_xlm_r.to_csv("/kaggle/working/results_translate_train_es_xlm_r.csv")

df_results_translate_train_es_xlm_r

lang  F1_Translate-train es XLM-R  EM_Translate-train es XLM-R
0  de.de                        62.65                        45.56
1  en.en                        79.05                        64.10
2  es.es                        69.35                        46.11
3  hi.hi                        62.61                        43.84
4    avg                        68.41                        49.90

<!-- split = "test"

results_translate_train_es_xlm_r = compute_results(langs_translate_train, split)

print(results_translate_train_es_xlm_r)

df_results_translate_train_es_xlm_r = results_df(results_translate_train_es_xlm_r, "Translate-train es XLM-R")

df_results_translate_train_es_xlm_r.to_csv("/kaggle/working/results_translate_train_es_xlm_r.csv")

df_results_translate_train_es_xlm_r -->

In [20]:
split = "test"

map_datasets(langs_test, split, prepare_validation_features)

Map:   0%|          | 0/4517 [00:00<?, ? examples/s]

Map:   0%|          | 0/1776 [00:00<?, ? examples/s]

Map:   0%|          | 0/1430 [00:00<?, ? examples/s]

Map:   0%|          | 0/4517 [00:00<?, ? examples/s]

Map:   0%|          | 0/5253 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/5253 [00:00<?, ? examples/s]

Map:   0%|          | 0/1723 [00:00<?, ? examples/s]

Map:   0%|          | 0/1430 [00:00<?, ? examples/s]

Map:   0%|          | 0/4918 [00:00<?, ? examples/s]

Map:   0%|          | 0/1723 [00:00<?, ? examples/s]

In [21]:
split = "test"

results_translate_train_es_xlm_r_ = compute_results(langs_test, split)

print(results_translate_train_es_xlm_r_)

df_results_translate_train_es_xlm_r_ = results_df(results_translate_train_es_xlm_r_, "Translate-train es XLM-R")

df_results_translate_train_es_xlm_r_.to_csv("/kaggle/working/results_translate_train_es_xlm_r_allpairs.csv")

df_results_translate_train_es_xlm_r_

Post-processing 4517 example predictions split into 5278 features.


  0%|          | 0/4517 [00:00<?, ?it/s]

Post-processing 4517 example predictions split into 5274 features.


  0%|          | 0/4517 [00:00<?, ?it/s]

Post-processing 1776 example predictions split into 2066 features.


...

  0%|          | 0/1723 [00:00<?, ?it/s]

Post-processing 4918 example predictions split into 6351 features.


  0%|          | 0/4918 [00:00<?, ?it/s]

{'de.de': {'exact_match': 45.56121319459818, 'f1': 62.6523863691191}, 'de.en': {'exact_match': 44.18862076599513, 'f1': 60.75102300504432}, 'de.es': {'exact_match': 42.567567567567565, 'f1': 57.44324328768438}, 'de.hi': {'exact_match': 27.132867132867133, 'f1': 40.794328118759665}, 'en.de': {'exact_match': 52.092096524241754, 'f1': 65.93839658594773}, 'en.en': {'exact_match': 64.09836065573771, 'f1': 79.04587623267263}, 'en.es': {'exact_match': 56.91985532076909, 'f1': 70.96497540890277}, 'en.hi': {'exact_match': 42.17161447742985, 'f1': 55.726686609438474}, 'es.de': {'exact_match': 38.23198198198198, 'f1': 60.04951041722269}, 'es.en': {'exact_match': 43.36569579288026, 'f1': 66.4937032638854}, 'es.es': {'exact_match': 46.10698648391396, 'f1': 69.34885176860061}, 'es.hi': {'exact_match': 28.670922809053977, 'f1': 48.253631676488496}, 'hi.de': {'exact_match': 33.63636363636363, 'f1': 49.34441738635788}, 'hi.en': {'exact_match': 41.98861325742172, 'f1': 60.0808131936454}, 'hi.es': {'exac

lang  F1_Translate-train es XLM-R  EM_Translate-train es XLM-R
0   de.de                        62.65                        45.56
1   de.en                        60.75                        44.19
2   de.es                        57.44                        42.57
3   de.hi                        40.79                        27.13
4   en.de                        65.94                        52.09
5   en.en                        79.05                        64.10
6   en.es                        70.96                        56.92
7   en.hi                        55.73                        42.17
8   es.de                        60.05                        38.23
9   es.en                        66.49                        43.37
10  es.es                        69.35                        46.11
11  es.hi                        48.25                        28.67
12  hi.de                        49.34                        33.64
13  hi.en                        60.08                        41.99
14  hi.es                        54.02                        37.78
15  hi.hi                        62.61                        43.84
16    avg                        60.22                        43.02

In [22]:
!zip -r /kaggle/working/checkpoint-7983.zip /kaggle/working/xlmr-base-texas-squad-es/checkpoint-7983


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/xlmr-base-texas-squad-es/checkpoint-7983/ (stored 0%)
  adding: kaggle/working/xlmr-base-texas-squad-es/checkpoint-7983/special_tokens_map.json (deflated 85%)
  adding: kaggle/working/xlmr-base-texas-squad-es/checkpoint-7983/tokenizer.json (deflated 76%)
  adding: kaggle/working/xlmr-base-texas-squad-es/checkpoint-7983/sentencepiece.bpe.model (deflated 49%)
  adding: kaggle/working/xlmr-base-texas-squad-es/checkpoint-7983/tokenizer_config.json (deflated 74%)
  adding: kaggle/working/xlmr-base-texas-squad-es/checkpoint-7983/training_args.bin (deflated 51%)
  adding: kaggle/working/xlmr-base-texas-squad-es/checkpoint-7983/model.safetensors (deflated 22%)
  adding: kaggle/working/xlmr-base-texas-squad-es/checkpoint-7983/rng_state.pth (deflated 25%)
  adding: kaggle/working/xlmr-base-texas-squad-es/checkpoint-7983/config.json (deflated 48%)
  adding: kaggle/working/xlmr-base-texas-squad-es/checkpoint-7983/optimizer.pt (deflated 63%)
  adding: kaggle/working/xlmr-ba